In [1]:
# 1. Imports pour le NLP et Deep Learning
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import numpy as np

# 2. Chargement des données (Nous allons nous concentrer uniquement sur le texte)
df_nlp = pd.read_csv('simulated_email_data_bilingual.csv')

# 3. Préparation pour Hugging Face (Dataset)
# Le NLP se concentrera uniquement sur le texte (subject + body)
df_nlp['text'] = df_nlp['subject'] + " [SEP] " + df_nlp['body']
df_nlp = df_nlp[['text', 'label']]

# 4. Séparation Entraînement/Test (Stratifié pour l'équilibre)
train_df, eval_df = train_test_split(df_nlp, 
                                     test_size=0.2, 
                                     random_state=42, 
                                     stratify=df_nlp['label'])

# 5. Conversion des DataFrames en objets Dataset Hugging Face
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
eval_dataset = Dataset.from_pandas(eval_df.reset_index(drop=True))

print("Datasets prêts.") 

Datasets prêts.


In [4]:
# 6. Définition du modèle bilingue
MODEL_NAME = "xlm-roberta-base" 

# 7. Initialisation du Tokenizer (conversion du texte en nombres pour le modèle)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 8. Fonction de Tokenisation
def tokenize_function(examples):
    # Tronque le texte à 512 tokens maximum (limite standard des Transformers)
    return tokenizer(examples["text"], truncation=True, padding="max_length")

# 9. Application de la Tokenisation
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# 10. Renommage de la colonne 'label' en 'labels' pour PyTorch (Exigence)
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_eval = tokenized_eval.rename_column("label", "labels")

# 11. Formatage final en PyTorch
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_eval = tokenized_eval.remove_columns(["text"])
tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")

print("Tokenisation terminée. Données prêtes pour le Deep Learning.")

Map:   0%|          | 0/1760 [00:00<?, ? examples/s]

Map:   0%|          | 0/440 [00:00<?, ? examples/s]

Tokenisation terminée. Données prêtes pour le Deep Learning.


In [5]:
# 12. Définition du Modèle de Classification
MODEL_NAME = "xlm-roberta-base" 
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# 13. Définition des Métriques d'Évaluation (Focus sur le F1-Score)
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', pos_label=1)

    return {
        'f1_score_BEC': f1,
        'precision_BEC': precision,
        'recall_BEC': recall,
    }

print("Modèle XLM-RoBERTa chargé.")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modèle XLM-RoBERTa chargé.


In [6]:
# 14. Configuration des Arguments d'Entraînement (VERSION LA PLUS STABLE ET ROBUSTE)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,

    # Arguments stabilisés après le diagnostic du ValueError
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=200,             
    save_steps=200,             
    save_total_limit=1,

    load_best_model_at_end=True,
    metric_for_best_model="f1_score_BEC",
)

# 15. Initialisation du Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    # N.B. : `model`, `tokenized_train`, et `tokenized_eval` sont maintenant définis
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 16. Lancement de l'Entraînement (Démarrage du Deep Learning !)
print("Début de l'entraînement NLP (Peut être long sans GPU)...")
trainer.train()

C:\Users\jjk\AppData\Local\Temp\ipykernel_7984\4029517311.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Début de l'entraînement NLP (Peut être long sans GPU)...


C:\Users\jjk\miniconda3\envs\aibec\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,F1 Score Bec,Precision Bec,Recall Bec
200,0.023300,0.000203,1.000000,1.000000,1.000000
400,0.000100,0.000030,1.000000,1.000000,1.000000
600,0.000000,0.000015,1.000000,1.000000,1.000000


C:\Users\jjk\miniconda3\envs\aibec\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\jjk\miniconda3\envs\aibec\lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=660, training_loss=0.062055891870069456, metrics={'train_runtime': 12723.9261, 'train_samples_per_second': 0.415, 'train_steps_per_second': 0.052, 'total_flos': 1389226372300800.0, 'train_loss': 0.062055891870069456, 'epoch': 3.0})